# Poetry Generation with LSTM

NO text preprocessing is done in poetry as data is used as is

In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [3]:
import numpy as np
import string
import pandas as pd

In [4]:
#Check GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## DATA

In [6]:
with open("bruce-springsteen.txt", "r") as br:
    data = br.read()

In [7]:
data

'They come from everywhere\nA longing to be free\nThey come to join us here\nFrom sea to shining sea And they all have a dream\nAs people always will\nTo be safe and warm\nIn that shining city on the hill Some wanna slam the door\nInstead of opening the gate\nAw, let\'s turn this thing around\nBefore it gets too late \nIt\'s up to me and you\nLove can conquer hate\nI know this to be true\nThat\'s what makes us great \nDon\'t tell me a lie\nAnd sell it as a fact\nI\'ve been down that road before\nAnd I ain\'t goin\' back And don\'t you brag to me\nThat you never read a book\nI never put my faith\nIn a con man and his crooks I won\'t follow down that path\nAnd tempt the hands of fate\nAw, let\'s turn this thing around\nBefore it gets too late \nIt\'s up to me and you\nLove can conquer hate\nI know this to be true\nThat\'s what makes us great In the quiet of the night\nI lie here wide awake\nAnd I ask myself\nIs there a difference I can make? \nIt\'s up to me and you\nLove can conquer hat

In [8]:
data = data.replace('\n',' ')
data

'They come from everywhere A longing to be free They come to join us here From sea to shining sea And they all have a dream As people always will To be safe and warm In that shining city on the hill Some wanna slam the door Instead of opening the gate Aw, let\'s turn this thing around Before it gets too late  It\'s up to me and you Love can conquer hate I know this to be true That\'s what makes us great  Don\'t tell me a lie And sell it as a fact I\'ve been down that road before And I ain\'t goin\' back And don\'t you brag to me That you never read a book I never put my faith In a con man and his crooks I won\'t follow down that path And tempt the hands of fate Aw, let\'s turn this thing around Before it gets too late  It\'s up to me and you Love can conquer hate I know this to be true That\'s what makes us great In the quiet of the night I lie here wide awake And I ask myself Is there a difference I can make?  It\'s up to me and you Love can conquer hate I know this to be true That\'s

In [9]:
len(" ".join(data))

214777

## Tokenization

In [10]:
token = Tokenizer()
token.fit_on_texts(data.split())

In [11]:
token.word_index

{'the': 1,
 'i': 2,
 'a': 3,
 'and': 4,
 'you': 5,
 'in': 6,
 'to': 7,
 'me': 8,
 'my': 9,
 'on': 10,
 'your': 11,
 'of': 12,
 'that': 13,
 "don't": 14,
 'all': 15,
 'love': 16,
 'down': 17,
 'but': 18,
 'it': 19,
 'night': 20,
 'well': 21,
 'for': 22,
 'oh': 23,
 'just': 24,
 'out': 25,
 "i'm": 26,
 'this': 27,
 'with': 28,
 'got': 29,
 'baby': 30,
 'was': 31,
 'we': 32,
 'up': 33,
 'come': 34,
 'be': 35,
 'no': 36,
 'know': 37,
 'they': 38,
 'like': 39,
 'la': 40,
 'back': 41,
 'now': 42,
 'one': 43,
 "ain't": 44,
 'when': 45,
 'there': 46,
 'is': 47,
 'if': 48,
 'so': 49,
 'what': 50,
 'little': 51,
 'want': 52,
 'here': 53,
 'her': 54,
 'yeah': 55,
 'tonight': 56,
 'at': 57,
 'can': 58,
 'let': 59,
 "it's": 60,
 'girl': 61,
 "i'll": 62,
 'go': 63,
 'from': 64,
 'do': 65,
 'he': 66,
 'she': 67,
 'man': 68,
 'pay': 69,
 'li': 70,
 "there's": 71,
 'as': 72,
 'will': 73,
 'some': 74,
 'his': 75,
 'hey': 76,
 'heart': 77,
 'by': 78,
 'say': 79,
 'are': 80,
 'get': 81,
 "you're": 82,
 'r

In [12]:
token.word_counts

OrderedDict([('they', 92),
             ('come', 105),
             ('from', 56),
             ('everywhere', 3),
             ('a', 561),
             ('longing', 4),
             ('to', 378),
             ('be', 102),
             ('free', 15),
             ('join', 2),
             ('us', 30),
             ('here', 68),
             ('sea', 8),
             ('shining', 11),
             ('and', 561),
             ('all', 159),
             ('have', 27),
             ('dream', 22),
             ('as', 48),
             ('people', 4),
             ('always', 29),
             ('will', 48),
             ('safe', 4),
             ('warm', 3),
             ('in', 483),
             ('that', 174),
             ('city', 21),
             ('on', 247),
             ('the', 897),
             ('hill', 4),
             ('some', 48),
             ('wanna', 28),
             ('slam', 3),
             ('door', 14),
             ('instead', 3),
             ('of', 229),
             ('opening', 2)

In [13]:
encoded_text = token.texts_to_sequences(data.split())
encoded_text

[[38],
 [34],
 [64],
 [737],
 [3],
 [609],
 [7],
 [35],
 [224],
 [38],
 [34],
 [7],
 [958],
 [121],
 [53],
 [64],
 [409],
 [7],
 [303],
 [409],
 [4],
 [38],
 [15],
 [132],
 [3],
 [161],
 [72],
 [610],
 [124],
 [73],
 [7],
 [35],
 [611],
 [4],
 [738],
 [6],
 [13],
 [303],
 [168],
 [10],
 [1],
 [612],
 [74],
 [130],
 [739],
 [1],
 [242],
 [740],
 [12],
 [959],
 [1],
 [960],
 [613],
 [371],
 [225],
 [27],
 [190],
 [141],
 [243],
 [19],
 [614],
 [142],
 [280],
 [60],
 [33],
 [7],
 [8],
 [4],
 [5],
 [16],
 [58],
 [443],
 [496],
 [2],
 [37],
 [27],
 [7],
 [35],
 [146],
 [87],
 [50],
 [372],
 [121],
 [497],
 [14],
 [125],
 [8],
 [3],
 [209],
 [4],
 [961],
 [19],
 [72],
 [3],
 [373],
 [122],
 [109],
 [17],
 [13],
 [244],
 [243],
 [4],
 [2],
 [44],
 [615],
 [41],
 [4],
 [14],
 [5],
 [962],
 [7],
 [8],
 [13],
 [5],
 [100],
 [741],
 [3],
 [498],
 [2],
 [100],
 [176],
 [9],
 [410],
 [6],
 [3],
 [963],
 [68],
 [4],
 [75],
 [964],
 [2],
 [374],
 [742],
 [17],
 [13],
 [550],
 [4],
 [551],
 [1],
 [198

In [14]:
len(encoded_text)

21454

In [15]:
encoded_text2 =[]
[encoded_text2.append(i[0]) for i in encoded_text if len(i)>0 ]
len(encoded_text2) 

21453

In [16]:
encoded_text2

[38,
 34,
 64,
 737,
 3,
 609,
 7,
 35,
 224,
 38,
 34,
 7,
 958,
 121,
 53,
 64,
 409,
 7,
 303,
 409,
 4,
 38,
 15,
 132,
 3,
 161,
 72,
 610,
 124,
 73,
 7,
 35,
 611,
 4,
 738,
 6,
 13,
 303,
 168,
 10,
 1,
 612,
 74,
 130,
 739,
 1,
 242,
 740,
 12,
 959,
 1,
 960,
 613,
 371,
 225,
 27,
 190,
 141,
 243,
 19,
 614,
 142,
 280,
 60,
 33,
 7,
 8,
 4,
 5,
 16,
 58,
 443,
 496,
 2,
 37,
 27,
 7,
 35,
 146,
 87,
 50,
 372,
 121,
 497,
 14,
 125,
 8,
 3,
 209,
 4,
 961,
 19,
 72,
 3,
 373,
 122,
 109,
 17,
 13,
 244,
 243,
 4,
 2,
 44,
 615,
 41,
 4,
 14,
 5,
 962,
 7,
 8,
 13,
 5,
 100,
 741,
 3,
 498,
 2,
 100,
 176,
 9,
 410,
 6,
 3,
 963,
 68,
 4,
 75,
 964,
 2,
 374,
 742,
 17,
 13,
 550,
 4,
 551,
 1,
 198,
 12,
 616,
 613,
 371,
 225,
 27,
 190,
 141,
 243,
 19,
 614,
 142,
 280,
 60,
 33,
 7,
 8,
 4,
 5,
 16,
 58,
 443,
 496,
 2,
 37,
 27,
 7,
 35,
 146,
 87,
 50,
 372,
 121,
 497,
 6,
 1,
 617,
 12,
 1,
 20,
 2,
 209,
 53,
 499,
 552,
 4,
 2,
 444,
 411,
 47,
 46,
 3,
 743,
 2

In [17]:
vocab_size = len(token.word_counts) + 1
vocab_size

2535

## Data preparation

In [18]:
length = 10
batch = 1
generator = TimeseriesGenerator(encoded_text2, encoded_text2, length=length, batch_size=batch)

In [19]:
X =[]
y =[]
for i,j in generator:
    X.append(i.reshape(10,))
    y.append(j.reshape(1,))
X =  np.array(X)   
y =  np.array(y)   

In [20]:
X.shape

(21443, 10)

In [21]:
X

array([[ 38,  34,  64, ...,  35, 224,  38],
       [ 34,  64, 737, ..., 224,  38,  34],
       [ 64, 737,   3, ...,  38,  34,   7],
       ...,
       [443, 496,   2, ..., 146,  87,  50],
       [496,   2,  37, ...,  87,  50, 372],
       [  2,  37,  27, ...,  50, 372, 121]])

In [22]:
y.shape

(21443, 1)

In [23]:
y

array([[ 34],
       [  7],
       [958],
       ...,
       [372],
       [121],
       [497]])

In [24]:
y = to_categorical(y, num_classes=vocab_size)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [26]:
y_train.shape

(17154, 2535)

In [27]:
X_train.shape

(17154, 10)

## LSTM Model Training

In [28]:
seq_length = X_train.shape[1]

In [34]:
features =300
model = Sequential()
model.add(Embedding(vocab_size, features, input_length=seq_length))
model.add(LSTM(100, activation='relu', input_shape=(seq_length, features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100 , return_sequences=True , activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(100 ,  activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 300)           760500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 100)           80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)              

In [44]:
model.compile(optimizer=Adam(learning_rate=0.9), loss='categorical_crossentropy',  metrics=['accuracy'])

In [42]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [46]:
model.fit(x = X_train,
          y = y_train,
          epochs=100,
          validation_data=(X_test,y_test),
          batch_size=32
          #callbacks=[early_stop]
                   )

Train on 17154 samples, validate on 4289 samples
Epoch 1/100
17154/17154 [==============================] - 11s 655us/sample - loss: 7.4493 - accuracy: 0.0270 - val_loss: 8.4065 - val_accuracy: 0.0315
Epoch 2/100
17154/17154 [==============================] - 11s 650us/sample - loss: 7.4738 - accuracy: 0.0256 - val_loss: 8.4807 - val_accuracy: 0.0224
Epoch 3/100
17154/17154 [==============================] - 11s 644us/sample - loss: 7.4647 - accuracy: 0.0266 - val_loss: 8.4445 - val_accuracy: 0.0224
Epoch 4/100
17154/17154 [==============================] - 11s 645us/sample - loss: 7.4686 - accuracy: 0.0300 - val_loss: 8.4274 - val_accuracy: 0.0224
Epoch 5/100
17154/17154 [==============================] - 11s 650us/sample - loss: 7.4545 - accuracy: 0.0297 - val_loss: 8.6200 - val_accuracy: 0.0058
Epoch 6/100
17154/17154 [==============================] - 11s 649us/sample - loss: 7.4591 - accuracy: 0.0261 - val_loss: 8.5780 - val_accuracy: 0.0210
Epoch 7/100
17154/17154 [==============

Epoch 54/100
17154/17154 [==============================] - 11s 667us/sample - loss: 7.4642 - accuracy: 0.0291 - val_loss: 8.7846 - val_accuracy: 0.0100
Epoch 55/100
17154/17154 [==============================] - 11s 669us/sample - loss: 7.4675 - accuracy: 0.0277 - val_loss: 8.7648 - val_accuracy: 0.0224
Epoch 56/100
17154/17154 [==============================] - 11s 668us/sample - loss: 7.4598 - accuracy: 0.0278 - val_loss: 8.9179 - val_accuracy: 0.0210
Epoch 57/100
17154/17154 [==============================] - 11s 664us/sample - loss: 7.4648 - accuracy: 0.0284 - val_loss: 8.7763 - val_accuracy: 0.0224
Epoch 58/100
17154/17154 [==============================] - 11s 663us/sample - loss: 7.4609 - accuracy: 0.0296 - val_loss: 8.7748 - val_accuracy: 0.0445
Epoch 59/100
17154/17154 [==============================] - 12s 677us/sample - loss: 7.4511 - accuracy: 0.0281 - val_loss: 8.9163 - val_accuracy: 0.0098
Epoch 60/100
17154/17154 [==============================] - 12s 682us/sample - los

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
losses = pd.DataFrame(model.history.history)

losses.plot()